In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import librosa

import sys

sys.path.append("./../../unilm/wavlm/")
from WavLM import WavLM, WavLMConfig

In [2]:
fs = 16000
ckpt_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/wavlm/WavLM-Large.pt"

checkpoint = torch.load(ckpt_path)
cfg = WavLMConfig(checkpoint['cfg'])
model = WavLM(cfg)
model.load_state_dict(checkpoint['model'])
model.eval();

In [3]:
class WavLMExtractor():
    def __init__(self, fs=16000, ckpt_path="/mntcephfs/lab_data/shoinoue/Models/trained_models/wavlm/WavLM-Large.pt", device="cuda"):
        self.fs = fs
        checkpoint = torch.load(ckpt_path)
        cfg = WavLMConfig(checkpoint['cfg'])
        model = WavLM(cfg)
        model.load_state_dict(checkpoint['model'])
        model.eval();
        self.model = model.to(device)
        self.cfg = cfg
        self.device = device
    
    def get_feature(self, path):
        wav, _ = librosa.load(path, self.fs)
        wav_input_16khz = torch.tensor(wav).unsqueeze(0)
        if self.cfg.normalize:
            wav_input_16khz = torch.nn.functional.layer_norm(wav_input_16khz , wav_input_16khz.shape)
        rep = self.model.extract_features(wav_input_16khz.to(self.device))[0][0]
        return rep

In [ ]:
path = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/HKK/wav/arctic_a0001.wav"

In [36]:
wavlm = WavLMExtractor()

In [38]:
wavlm.get_feature(path).shape

torch.Size([179, 1024])

In [20]:
# extract the representation of last layer


In [21]:
rep

tensor([[-0.0630,  0.1153, -0.1073,  ..., -0.0424,  0.3143, -0.2737],
        [ 0.1324,  0.0528, -0.2474,  ...,  0.1291,  0.0365, -0.1063],
        [ 0.1097, -0.0708, -0.0916,  ...,  0.1428, -0.0264,  0.2364],
        ...,
        [ 0.0769,  0.2265,  0.0669,  ..., -0.0678,  0.0518,  0.1354],
        [-0.0420,  0.1428,  0.0902,  ..., -0.1188,  0.0996,  0.2102],
        [-0.2314, -0.0065,  0.1919,  ...,  0.0016,  0.1852,  0.0608]],
       grad_fn=<SelectBackward>)

In [10]:
rep.shape

torch.Size([1, 31, 1024])

In [6]:
rep

tensor([[[-35.3634,  28.3430, -93.9248,  ..., -63.4952,  60.0124,  18.8887],
         [ -1.6355,  40.0090, -85.3306,  ..., -64.3624,  33.2331, -28.8264],
         [-10.1995,  33.1479, -71.1561,  ..., -67.3214,  24.3543, -33.4058],
         ...,
         [-72.8382,  33.7898,   5.8237,  ..., -71.1066, -19.8455, -42.0761],
         [-66.8966,  33.8302,   5.1525,  ..., -76.2053, -11.9666, -53.3977],
         [-78.6402,  18.9396, -19.4920,  ..., -38.9124,  18.8453,  -7.0787]]],
       grad_fn=<TransposeBackward0>)

In [ ]:
wav_